![copcarimage](https://timelinecovers.pro/facebook-cover/download/police-car-in-action-facebook-cover.jpg)

***

<hr style="height:5px;border-width:0;color:orange;background-color:orange">
<center> <span style="font-size: 26px;"> Unsupervised Machine Learning Assignment </span> </center>

<span style="font-size: 16px;">

+ Session: **Unsupervised Machine Learning **
+ Assignment: ***Crimes in Boston***
+ Submitted by: **Paul Jason Jumalon**
+ Submitted to: **Christa Santos**



</span>

<br>

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

## Import and Install

In [ ]:
import numpy as np # linear algebra
import pandas as pd # Series and DataFrames

# import libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.2f}'.format

from matplotlib import pyplot as plt
import plotly.express as px

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

from sklearn import preprocessing

import missingno as msno

from prophet import Prophet

In [ ]:
#pip install Prophet

In [ ]:
#pip install missingno

In [ ]:
#from prophet import Prophet


In [ ]:
df1 = pd.read_csv('crime.csv', encoding = "latin-1")

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.describe()

In [ ]:
df1.info()

## Cleaning the data

In [ ]:
backup_df = df1.copy()

In [ ]:
df1.select_dtypes(include=['object']).describe().T


In [ ]:
df1.drop_duplicates(subset="INCIDENT_NUMBER", inplace=True)


In [ ]:
df1.select_dtypes(include=['number']).describe().T


In [ ]:
df1.select_dtypes(include=['object']).isnull().sum()

In [ ]:
df1.select_dtypes(include=['number']).isnull().sum()

Visual graph of Null values in Data frame

In [ ]:
msno.matrix(df1);

In [ ]:
df1.head()

In [ ]:
df1["SHOOTING"].fillna("N", inplace = True)
df1["DISTRICT"].fillna("Unknown", inplace = True)
df1["STREET"].fillna("Unknown", inplace = True)


In [ ]:
df1['Lat'] = df1['Lat'].replace(np.nan, df1['Lat'].median())
df1['Long'] = df1['Long'].replace(np.nan, df1['Long'].median())
df1['Lat'] = df1['Lat'].replace(-1, df1['Lat'].median())
df1['Long'] = df1['Long'].replace(-1, df1['Long'].median())

#### All NULLs / Duplicates and Longitude / Lattitude Outliers have been treated

# Analyzing the data, data visualizations 

In [ ]:
df1['OFFENSE_CODE_GROUP'].value_counts(dropna=np.False_)

In [ ]:
print(plt.style.available)

In [ ]:
#pip install mpldatacursor 

## Crimes by Type and Location 

In [ ]:
sort = df1['OFFENSE_CODE_GROUP'].value_counts().head(3).index

sns.countplot(data = df1, x = 'OFFENSE_CODE_GROUP', hue = 'DISTRICT', order = sort, palette = 'rainbow')
plt.title('Top 3 Crime Group by District')
fig = plt.gcf()
fig.set_size_inches(11,8)
plt.legend(loc='upper right', prop={'size': 7});



In [ ]:
df1['DISTRICT'].value_counts(dropna=np.False_)

In [ ]:
sort = df1['DISTRICT'].value_counts().head(3).index

sns.countplot(data = df1, x = 'DISTRICT', hue = 'OFFENSE_CODE_GROUP', order = sort, palette = 'rainbow')
plt.title('Districts with Highest Crime rates by Day ')
plt.title('Top 3 District by Crime Group')
fig = plt.gcf()
fig.set_size_inches(35,8)

plt.legend(loc= 'upper right', prop={'size': 7});


In [ ]:
sort = df1['DISTRICT'].value_counts().head(5).index
plt.style.use('bmh')

sns.countplot(data = df1, x = 'DISTRICT', hue = 'DAY_OF_WEEK', order = sort, palette = 'rainbow')
plt.title('Districts with Highest Crime rates by Day of the week ')
fig = plt.gcf()
fig.set_size_inches(11,8)
plt.legend(loc='upper right', prop={'size': 7});

In [ ]:
sort = df1['OFFENSE_CODE_GROUP'].value_counts().index

sns.countplot(data = df1, x = 'OFFENSE_CODE_GROUP', order = sort, palette = 'rainbow')
plt.title('Crime types in Boston,Jun 2015 to Sep 2018')

fig = plt.gcf()
fig.set_size_inches(18,8)

plt.xticks(
    rotation=90,
    horizontalalignment='right',
    fontweight='light',
    fontsize='medium',
)

********************

## Crimes with Shootings

In [ ]:
Crimeswthshooting = pd.DataFrame(data =(df1.groupby(['DISTRICT','SHOOTING']).count()[['INCIDENT_NUMBER']]).reset_index().values, columns=["DISTRICT","SHOOTING","CRIME COUNT",]).sort_values('SHOOTING').reset_index(drop=True)
ShootingbyCrime = pd.DataFrame(data =(df1.groupby(['OFFENSE_CODE_GROUP','SHOOTING']).count()[['INCIDENT_NUMBER']]).reset_index().values, columns=['OFFENSE_CODE_GROUP',"SHOOTING","CRIME COUNT",]).sort_values('SHOOTING').reset_index(drop=True)


In [ ]:
fig = px.pie(Crimeswthshooting, values='CRIME COUNT', names='SHOOTING',title='Crimes with SHOOTINGS')
fig.show()

DESELECT SHOOTING = N, in Legend, to see exact Shooting numbers by Category

In [ ]:
px.bar(Crimeswthshooting, x='DISTRICT', y="CRIME COUNT", color="SHOOTING",barmode='group', title='Crimes per districts with SHOOTINGS')


DESELECT SHOOTING = N, in Legend, to see exact Shooting numbers by Category

In [ ]:
px.bar(Crimeswthshooting, x='DISTRICT', y="CRIME COUNT", color="SHOOTING",barmode='group', title='Crimes per districts with SHOOTINGS')


DESELECT SHOOTING = N, in Legend, to see exact Shooting numbers by Category

In [ ]:
px.bar(ShootingbyCrime, x='OFFENSE_CODE_GROUP', y="CRIME COUNT", color="SHOOTING",barmode='group', title='Crimes per districts with SHOOTINGS')


## Analyzing the data over different time breakdowns 

In [ ]:
m = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

ucrbyday = pd.DataFrame(data =(df1.groupby(['DAY_OF_WEEK','UCR_PART']).count()[['INCIDENT_NUMBER']]).reset_index().values, columns=["DAY","UCR_PART","CRIME COUNT"]).sort_values('DAY').reset_index(drop=True)
ucrbyday['DAY'] = pd.Categorical(ucrbyday['DAY'], categories=m, ordered=True)
ucrbyday = ucrbyday.sort_values('DAY').reset_index(drop=True)
ucrbydhour = pd.DataFrame(data =(df1.groupby(['HOUR','UCR_PART']).count()[['INCIDENT_NUMBER']]).reset_index().values, columns=["HOUR","UCR_PART","CRIME COUNT"]).sort_values('HOUR').reset_index(drop=True)

In [ ]:
px.bar(ucrbyday, x='DAY', y="CRIME COUNT", color="UCR_PART",title='Crime counts by Day across UCR Parts')


In [ ]:
px.bar(ucrbyday, x='UCR_PART', y='CRIME COUNT', color = 'DAY',title='Crime counts by Day across UCR Parts')


In [ ]:
m = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]


df1['OCCURRED_ON_DATE'] = pd.to_datetime(df1['OCCURRED_ON_DATE'])

# Creating two new features
df1["Quarter"] = df1["OCCURRED_ON_DATE"].dt.quarter
df1["Weekofyear"] = df1["OCCURRED_ON_DATE"].dt.weekofyear

# Convert into categorical data type
df1["Quarter"] = df1["Quarter"].astype("category")
df1["Weekofyear"] = df1["Weekofyear"].astype("category")

In [ ]:
crimes_per_year = pd.DataFrame(data=df1['YEAR'].value_counts().reset_index().values, columns=["YEAR","CRIME COUNT"]).sort_values('YEAR').reset_index(drop=True)
crimes_per_month = pd.DataFrame(data=df1['MONTH'].value_counts().reset_index().values, columns=["MONTH","CRIME COUNT"]).sort_values('MONTH').reset_index(drop=True)
crimes_per_day = pd.DataFrame(data=df1['DAY_OF_WEEK'].value_counts().reset_index().values, columns=["DAY","CRIME COUNT"])
crimes_per_day["DAY"] = pd.Categorical(crimes_per_day['DAY'], categories=m, ordered=True)
crimes_per_day = crimes_per_day.sort_values('DAY').reset_index(drop=True)
crimes_per_hour = pd.DataFrame(data=df1['HOUR'].value_counts().reset_index().values, columns=["HOUR","CRIME COUNT"]).sort_values('HOUR').reset_index(drop=True)
crimes_per_Weekofyear = pd.DataFrame(data=df1['Weekofyear'].value_counts().reset_index().values, columns=["Weekofyear","CRIME COUNT"])
crimes_per_Quarter = pd.DataFrame(data=df1['Quarter'].value_counts().reset_index().values, columns=["Quarter","CRIME COUNT"]).sort_values('CRIME COUNT',ascending=False).reset_index(drop=True).head(50)

In [ ]:
offense_code_count = pd.DataFrame(df1['OFFENSE_CODE_GROUP'].value_counts().reset_index().values, columns=["OFFENSE_CODE_GROUP'","CRIME COUNT"])


In [ ]:
df1.columns

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=3,specs=[[{"type": "scatter"}, {"type": "scatter"},{"type": "scatter"}],[{"type": "scatter"},{"type": "bar"}, {"type": "bar"}]],
    subplot_titles=("Number of crimes per year", "Number of crimes per month", "Number of crimes per day", "Number of crimes per hour","Number of crimes per Week of year","Number of crimes per Quarter"))
# Add traces
fig.add_trace(go.Scatter(x=crimes_per_year["YEAR"], y=crimes_per_year["CRIME COUNT"]), row=1, col=1)
fig.add_trace(go.Scatter(x=crimes_per_month["MONTH"], y=crimes_per_month["CRIME COUNT"]), row=1, col=2)
fig.add_trace(go.Bar(x=crimes_per_day["DAY"], y=crimes_per_day["CRIME COUNT"]), row=1, col=3)
fig.add_trace(go.Scatter(x=crimes_per_hour["HOUR"], y=crimes_per_hour["CRIME COUNT"]), row=2, col=1)
fig.add_trace(go.Bar(x=crimes_per_Weekofyear["Weekofyear"], y=crimes_per_Weekofyear["CRIME COUNT"]), row=2, col=2)
fig.add_trace(go.Bar(x=crimes_per_Quarter["Quarter"], y=crimes_per_Quarter["CRIME COUNT"]), row=2, col=3)
# Update xaxis properties
fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_xaxes(title_text="Month", range=[0, 13], row=1, col=2)
fig.update_xaxes(title_text="Day", row=1, col=3)
fig.update_xaxes(title_text="Hour",row=2, col=1)
fig.update_xaxes(title_text="Weekofyear", row=2, col=2)
fig.update_xaxes(title_text="Quarter", row=2, col=3)
# Update yaxis properties
fig.update_yaxes(title_text="Crime Count", row=1, col=1)
fig.update_yaxes(title_text="Crime Count",row=1, col=2)
fig.update_yaxes(title_text="Crime Count", row=1, col=3)
fig.update_yaxes(title_text="Crime Count", row=2, col=1)
fig.update_yaxes(title_text="Crime Count", row=2, col=2)
fig.update_yaxes(title_text="Crime Count", row=2, col=3)
# Update title
fig.update_layout(showlegend=False,title_text="Total Crimes in Boston broken down in TIME", height=900)
fig.show()

******************

df1.OFFENSE_CODE_GROUP = df1.OFFENSE_CODE_GROUP.astype('category')
df1.OFFENSE_DESCRIPTION = df1.OFFENSE_DESCRIPTION.astype('category')
df1.DAY_OF_WEEK = df1.DAY_OF_WEEK.astype('category')
df1.UCR_PART = df1.UCR_PART.astype('category')
df1.DISTRICT = df1.DISTRICT.astype('category')

In [ ]:
df1.dtypes

In [ ]:
df1.head()

In [ ]:
df1.columns

In [ ]:
monthly_crimes = df1[['MONTH','INCIDENT_NUMBER']].groupby(['MONTH']).count()
crimesbyhour = df1[['HOUR','INCIDENT_NUMBER']].groupby(['HOUR']).count()
yearly_crimes = df1[['YEAR','INCIDENT_NUMBER']].groupby(['YEAR']).count()
crimesbyday = df1[['DAY_OF_WEEK','INCIDENT_NUMBER']].groupby(['DAY_OF_WEEK']).count()
crimesbyhour = df1[['HOUR','INCIDENT_NUMBER']].groupby(['HOUR']).count()
offenses = df1[['OFFENSE_CODE_GROUP','INCIDENT_NUMBER']].groupby(['OFFENSE_CODE_GROUP']).count()
daily_crimes = df1[['OCCURRED_ON_DATE','INCIDENT_NUMBER']].groupby(['OCCURRED_ON_DATE']).count()


In [ ]:
df1.head()

## Crimes by Trend 

In [ ]:
from mpldatacursor import datacursor

crhour = crimesbyhour.plot(figsize=(15, 5))
datacursor(crhour, display='multiple', draggable=True)
plt.show()



In [ ]:
sort = df1['DISTRICT'].value_counts().head(3).index
plt.style.use('bmh')

sns.countplot(data = df1, x = 'DISTRICT', hue = 'HOUR', order = sort, palette='rainbow')
plt.title('Districts with Highest Crime rates by Hour ')
fig = plt.gcf()
fig.set_size_inches(11,8)
plt.legend(loc='upper right', prop={'size': 7});

In [ ]:
monthly_crimes.plot(figsize=(15, 5))
plt.show()

Crime trend matches the Climate trend of Boston 2017, there are more crimes during hotter climates that colder

https://weatherspark.com/h/y/26197/2017/Historical-Weather-during-2017-in-Boston-Massachusetts-United-States#Figures-Temperature

***

In [ ]:
sort = df1['DISTRICT'].value_counts().head(3).index
plt.style.use('bmh')

sns.countplot(data = df1, x = 'DISTRICT',  hue = 'MONTH', order = sort, palette='rainbow')
plt.title('Districts with Highest Crime rates by MONTH ')
fig = plt.gcf()
fig.set_size_inches(11,8)
plt.legend(loc='upper right', prop={'size': 7});

In [ ]:
yearly_crimes.plot(figsize=(15, 5))
plt.show()

In [ ]:
sort = df1['DISTRICT'].value_counts().head(3).index

sns.countplot(data = df1, x = 'DISTRICT', hue = 'YEAR', order = sort, palette='rainbow')
plt.title('Districts with Highest Crime rates by YEAR ')
fig = plt.gcf()
fig.set_size_inches(11,8)
plt.legend(loc='upper right', prop={'size': 7});

In [ ]:
fig, ax = plt.subplots(figsize=(50,18))
sns.lineplot(x='OCCURRED_ON_DATE', y='INCIDENT_NUMBER', data=daily_crimes,
             ax=ax).set_title("Daily Boston Crime numbers")

In [ ]:
sns.pairplot(df1, hue='DISTRICT')


In [ ]:
#pip install dash

In [ ]:
import os.path
import sys, json
import requests
import subprocess

import plotly.express as px
import matplotlib.pyplot as plt
from requests.exceptions import RequestException
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from collections import namedtuple

In [ ]:
#pip install geopandas
#pip install shapely


Use map below to look into specific streets and addresses, this map will allow you to see if a crime happend, when a crime happened and what kind of crime happened. 

In [ ]:
fig_map_box = px.scatter_mapbox(df1,
                        lat='Lat',
                        lon='Long',
                        hover_data =['OFFENSE_CODE', 'OFFENSE_DESCRIPTION', 'OCCURRED_ON_DATE'],
                        color='OFFENSE_CODE_GROUP',
                        #size="OFFENSE_CODE", color_continuous_scale=px.colors.sequential.Rainbow, size_max=100,
                        size_max=80,
                        zoom = 8,
                        mapbox_style='open-street-map')
fig_map_box.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})
fig_map_box.show()

# Holt Winters' Seasonal Time series Model 

## Prepare Your Dataset for Modeling


In [ ]:
df1.columns

In [ ]:
daily_crimes

In [ ]:
#daily_crimes['OCCURRED_ON_DATE'] = pd.to_datetime(df1['OCCURRED_ON_DATE'])
# Set the date as index 
#df2 = df1.set_index('OCCURRED_ON_DATE') -- already set in daily_crimes
# Select the proper time period for weekly aggreagation
daily_crimes1 = daily_crimes['2015-06-15':'2018-08-26'].resample('W').sum()
daily_crimes1.head()

In [ ]:
px.line(daily_crimes1, y = 'INCIDENT_NUMBER')

In [ ]:
import warnings
y = daily_crimes1['INCIDENT_NUMBER'] #defining y

fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(y,marker='.', linestyle='-', linewidth=0.5, label='Weekly')
ax.plot(y.resample('M').mean(),marker='o', markersize=8, linestyle='-', label='Monthly Mean Resample')
ax.set_ylabel('Crime Counts')
ax.legend();

## Decompose the Data


In [ ]:
import statsmodels.api as sm

# graphs to show seasonal_decompose
def seasonal_decompose (y):
    decomposition = sm.tsa.seasonal_decompose(y, model='additive',extrapolate_trend='freq')
    fig = decomposition.plot()
    fig.set_size_inches(14,7)
    plt.show()

In [ ]:
seasonal_decompose(y)

## Check for Stationarity


In [ ]:
### plot for Rolling Statistic for testing Stationarity
def test_stationarity(timeseries, title):
    
    #Determing rolling statistics
    rolmean = pd.Series(timeseries).rolling(window=12).mean() 
    rolstd = pd.Series(timeseries).rolling(window=12).std()
    
    fig, ax = plt.subplots(figsize=(16, 4))
    ax.plot(timeseries, label= title)
    ax.plot(rolmean, label='rolling mean');
    ax.plot(rolstd, label='rolling std (x10)');
    ax.legend()

In [ ]:
pd.options.display.float_format = '{:.8f}'.format
test_stationarity(y,'raw data')

In [ ]:
# Augmented Dickey-Fuller Test
from statsmodels.tsa.stattools import adfuller

def ADF_test(timeseries, dataDesc):
    print(' > Is the {} stationary ?'.format(dataDesc))
    dftest = adfuller(timeseries.dropna(), autolag='AIC')
    print('Test statistic = {:.3f}'.format(dftest[0]))
    print('P-value = {:.3f}'.format(dftest[1]))
    print('Critical values :')
    for k, v in dftest[4].items():
        print('\t{}: {} - The data is {} stationary with {}% confidence'.format(k, v, 'not' if v<dftest[0] else '', 100-int(k[:-1])))

In [ ]:
ADF_test(y,'raw data')

In [ ]:
# Detrending
y_detrend =  (y - y.rolling(window=12).mean())/y.rolling(window=12).std()

test_stationarity(y_detrend,'de-trended data')
ADF_test(y_detrend,'de-trended data')

## Differencing


In [ ]:
# Differencing
y_12lag =  y - y.shift(12)

test_stationarity(y_12lag,'12 lag differenced data')
ADF_test(y_12lag,'12 lag differenced data')

## Combining Detrending and Differencing

In [ ]:
# Detrending + Differencing

y_12lag_detrend =  y_detrend - y_detrend.shift(12)

test_stationarity(y_12lag_detrend,'12 lag differenced de-trended data')
ADF_test(y_12lag_detrend,'12 lag differenced de-trended data')

In [ ]:
daily_crimes

In [ ]:
y_to_train = y[:'2018-02-18'] # dataset to train
y_to_val = y['2018-02-25':] # last 6 months for test  
predict_date = len(y) - len(y[:'2018-02-25']) # the number of data points for the test set

### Holt-Winters’ Seasonal Method


In [ ]:
import numpy as np
from statsmodels.tsa.api import SimpleExpSmoothing 

def ses(y, y_to_train,y_to_val,smoothing_level,predict_date):
    y.plot(marker='o', color='black', legend=True, figsize=(14, 7))
    
    fit1 = SimpleExpSmoothing(y_to_train).fit(smoothing_level=smoothing_level,optimized=False)
    fcast1 = fit1.forecast(predict_date).rename(r'$\alpha={}$'.format(smoothing_level))
    # specific smoothing level
    fcast1.plot(marker='o', color='blue', legend=True)
    fit1.fittedvalues.plot(marker='o',  color='blue')
    mse1 = ((fcast1 - y_to_val) ** 2).mean()
    print('The Root Mean Squared Error of our forecasts with smoothing level of {} is {}'.format(smoothing_level,round(np.sqrt(mse1), 2)))
    
    ## auto optimization
    fit2 = SimpleExpSmoothing(y_to_train).fit()
    fcast2 = fit2.forecast(predict_date).rename(r'$\alpha=%s$'%fit2.model.params['smoothing_level'])
    # plot
    fcast2.plot(marker='o', color='green', legend=True)
    fit2.fittedvalues.plot(marker='o', color='green')
    
    mse2 = ((fcast2 - y_to_val) ** 2).mean()
    print('The Root Mean Squared Error of our forecasts with auto optimization is {}'.format(round(np.sqrt(mse2), 2)))
    
    plt.show()

In [ ]:
ses(y, y_to_train,y_to_val,0.8,predict_date)


In [ ]:
from statsmodels.tsa.api import Holt

def holt(y,y_to_train,y_to_val,smoothing_level,smoothing_slope, predict_date):
    y.plot(marker='o', color='black', legend=True, figsize=(14, 7))
    
    fit1 = Holt(y_to_train).fit(smoothing_level, smoothing_slope, optimized=False)
    fcast1 = fit1.forecast(predict_date).rename("Holt's linear trend")
    mse1 = ((fcast1 - y_to_val) ** 2).mean()
    print('The Root Mean Squared Error of Holt''s Linear trend {}'.format(round(np.sqrt(mse1), 2)))

    fit2 = Holt(y_to_train, exponential=True).fit(smoothing_level, smoothing_slope, optimized=False)
    fcast2 = fit2.forecast(predict_date).rename("Exponential trend")
    mse2 = ((fcast2 - y_to_val) ** 2).mean()
    print('The Root Mean Squared Error of Holt''s Exponential trend {}'.format(round(np.sqrt(mse2), 2)))
    
    fit1.fittedvalues.plot(marker="o", color='blue')
    fcast1.plot(color='blue', marker="o", legend=True)
    fit2.fittedvalues.plot(marker="o", color='red')
    fcast2.plot(color='red', marker="o", legend=True)

    plt.show()

In [ ]:
holt(y, y_to_train,y_to_val,0.6,0.2,predict_date)


In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing

def holt_win_sea(y,y_to_train,y_to_val,seasonal_type,seasonal_period,predict_date):
    
    y.plot(marker='o', color='black', legend=True, figsize=(14, 7))
    
    if seasonal_type == 'additive':
        fit1 = ExponentialSmoothing(y_to_train, seasonal_periods = seasonal_period, trend='add', seasonal='add').fit(use_boxcox=True)
        fcast1 = fit1.forecast(predict_date).rename('Additive')
        mse1 = ((fcast1 - y_to_val) ** 2).mean()
        print('The Root Mean Squared Error of additive trend, additive seasonal of '+ 
              'period season_length={} and a Box-Cox transformation {}'.format(seasonal_period,round(np.sqrt(mse1), 2)))
        
        fit2 = ExponentialSmoothing(y_to_train, seasonal_periods = seasonal_period, trend='add', seasonal='add', damped=True).fit(use_boxcox=True)
        fcast2 = fit2.forecast(predict_date).rename('Additive+damped')
        mse2 = ((fcast2 - y_to_val) ** 2).mean()
        print('The Root Mean Squared Error of additive damped trend, additive seasonal of '+ 
              'period season_length={} and a Box-Cox transformation {}'.format(seasonal_period,round(np.sqrt(mse2), 2)))
        
        fit1.fittedvalues.plot(style='--', color='red')
        fcast1.plot(style='--', marker='o', color='red', legend=True)
        fit2.fittedvalues.plot(style='--', color='green')
        fcast2.plot(style='--', marker='o', color='green', legend=True)
    
    elif seasonal_type == 'multiplicative':  
        fit3 = ExponentialSmoothing(y_to_train, seasonal_periods = seasonal_period, trend='add', seasonal='mul').fit(use_boxcox=True)
        fcast3 = fit3.forecast(predict_date).rename('Multiplicative')
        mse3 = ((fcast3 - y_to_val) ** 2).mean()
        print('The Root Mean Squared Error of additive trend, multiplicative seasonal of '+ 
              'period season_length={} and a Box-Cox transformation {}'.format(seasonal_period,round(np.sqrt(mse3), 2)))
        
        fit4 = ExponentialSmoothing(y_to_train, seasonal_periods = seasonal_period, trend='add', seasonal='mul', damped=True).fit(use_boxcox=True)
        fcast4 = fit4.forecast(predict_date).rename('Multiplicative+damped')
        mse4 = ((fcast3 - y_to_val) ** 2).mean()
        print('The Root Mean Squared Error of additive damped trend, multiplicative seasonal of '+ 
              'period season_length={} and a Box-Cox transformation {}'.format(seasonal_period,round(np.sqrt(mse4), 2)))
        
        fit3.fittedvalues.plot(style='--', color='red')
        fcast3.plot(style='--', marker='o', color='red', legend=True)
        fit4.fittedvalues.plot(style='--', color='green')
        fcast4.plot(style='--', marker='o', color='green', legend=True)
        
    else:
        print('Wrong Seasonal Type. Please choose between additive and multiplicative')

    plt.show()

In [ ]:
holt_win_sea

After Prophet installation this doesn't work anymore

In [ ]:
#holt_win_sea(y, y_to_train,y_to_val,'additive',52, predict_date)


# Prophet Prediction

In [ ]:
daily_crimes2 = daily_crimes
daily_crimes2.head()

In [ ]:
daily_crimes2 = daily_crimes['2015-06-15':'2018-08-26'].resample('W').sum()


In [ ]:
daily_crimes2 = daily_crimes2.reset_index()

In [ ]:
#prediction for Items sold over time
#daily_crimes2.index.rename('ds', inplace=True)
daily_crimes2.rename(columns={'INCIDENT_NUMBER': 'y'}, inplace=True)
daily_crimes2.rename(columns={'OCCURRED_ON_DATE': 'ds'}, inplace=True)

In [ ]:
daily_crimes2.head()

## Build and 'fit' (train) the model

In [ ]:
import logging
logging.getLogger('fbprophet').setLevel(logging.WARNING) 

In [ ]:
m = Prophet()
m.fit(daily_crimes2)


In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast

## Prophet Visualization of results : Prediction of Crimes in Boston

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

fig = plot_plotly(m, forecast)
fig.show()

we want to see the components of the forecast, we can use the Prophet.plot_components method. By default, you will see the trend, the annual seasonality and the weekly seasonality of the time series. If holidays are included, they will also appear on the chart.

In [ ]:
fig = plot_components_plotly(m, forecast)
fig.show()

We can also add holiday date information to Peyton's forecast, since we hypothesize changes in crime rates.

In [ ]:
m = Prophet()
m.add_country_holidays(country_name='US')
m.fit(daily_crimes2)

In [ ]:
m.train_holiday_names

In [ ]:
forecast = m.predict(future)
fig = plot_components_plotly(m, forecast)
fig.show()

### Fourier Order of Seasonalities
Seasonalities are estimated using a partial Fourier sum. The number of terms in the partial sum(the order) is a parameter that determines how quickly the seasonality can change.


The default Fourier order for yearly seasonality is 10. The default values are often appropriate, but they can be increased when the seasonality needs to fit higher-frequency changes, and generally be less smooth. The Fourier order can be specified for each built-in seasonality when instantiating the model, here it is increased to 20.

In [ ]:
from prophet.plot import plot_yearly
m_yearly = Prophet(yearly_seasonality=20).fit(daily_crimes2)
a = plot_yearly(m_yearly)

***
# CONCLUSIONS : 

Process :
* Data consists of the the following: 
	* Index(['INCIDENT_NUMBER', 'OFFENSE_CODE', 'OFFENSE_CODE_GROUP',
       'OFFENSE_DESCRIPTION', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING',
       'OCCURRED_ON_DATE', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'UCR_PART',
       'STREET', 'Lat', 'Long', 'Location', 'Quarter', 'Weekofyear'],

* Data Quality : 
	* MSNO Matrix shows the following Columns had NULL values : 'SHOOTING','DISTRICT', 'STREET', 'Lat' and 'Long'. 'INCIDENT_NUMBER', also had duplicates that needed cleaning

* Date Range : June 2015 to Sep 2018


Predictions :

*      Seasonality patterns from Jul 2018 to Jul 2019 will observe the same patterns as previous years but slightly lower in value. 
*      Foreseen spikes in Crime during New Year's and Drops during Christmas
*      Overall reduction towards July 2019

Conclusions
LOCATION, DISTRICT 
* -Top 3 Districts with the highest crime counts would be B2, C11, D4.
* -Top Crime in B2 and C11 is Motor Vehicle Accident Response while Top Crime in D4 is Larceny
* -Different districts may have different peak crime days within the week, for example : looking at the top 5 districts the peaks happen within the following days
*		-B2 - Wednesday, Thursday to Fridays 
*		-C11 - Mondays, Tuesdays and Wednesdays
*		-D4 -Friday and Wednesday
*		-B3 - Mondays, Tuesdays and Wednesdays
*		-A1- Friday and Saturdays
* Sunday is seen to be consistently low across all TOP 5 Districts
	
CRIMES with Shootings
*  	-Only 0.22 percent of the crimes have associated SHOOTINGS
*	-Highest number of Shootings happen in Districts B2 C11 and B3
*	-Crime Groups with Highest shootings are Aggrevated Assault and Homicide

UCR Parts
*	-Highest UCR part is Part 3 and Lowest is Other followed by Part 1. 
*	-UCR Parts are equally disctributed through the different days


CRIME Types 
* -Top crime in Boston is Motor Vehicle Accident Response which consists of 11.64 percent all crimes
*	-8 percent goes to Larceny
*	-7.38 percent falls under the offense code group of Medical assistance (there is an opportunity to define cause of Medical assistance related to a specific offense)

TIME (Year, Month, Day of the week, Hour, Week of the year, Quarter)
* In General: 
* Year, Graph shows a higher Crime count in 2017 compared to 2016, 2015 and 2018 however does not contain a full year's worth of data to make any assumptions
* Month, Highest crime counts occured July and August while the lowest was in February 
*             - The Crime rate seems to follow the same trend line as Boston's climate 
*             https://weatherspark.com/y/26197/Average-Weather-in-Boston-Massachusetts-United-States-Year-Round#Figures-Temperature
* Day of the week,
*	-Crime increases drastically from Thursday to Friday, Friday having the highest crime count while it drastically drops during the weekend, Sunday having the lowest crime count. 
* Hour, the hours with the highest crime rates are 15h, 16h, 17h being the highest, and 18h then drops from there until another crime spike at midnight. Lowest crime counts happen between 4am to 6am.
* Week of the year, For weekly planning of task force deployment; the Boston department can note that the weeks with the highest crime rates are between week 25 and week 35, the lowest crime happens on week 6 not counting the leap years
* Quarter 3 has the Highest Crime rate while Quarter 1 has the lowest

TIME versus DISTRICT 3 Districts
*	-Hour, The trend of crime generally follows the overall trend except for District B2 not seeing an increas during 12nn.
*	-Year and Month are aligned with overall trend
*	-Day of the Week crime counts may vary per district as shown in District analysis conclusion

CRIME TYPE ON THE MAP
* There is an available Map that lets you zoom into the streets and see and filter specific crime types
* There is no visual concentration of specific crime types plotted on the map 

PREDICTIONS : (additional KMEANS attempt below)
* Looking at the data showing seasonality in crime trends with Boston's climate
* Using Holt-Winters’ Seasonal Method, prediction falls under an additive type of seasonality. 
*      	Additive: xt = Trend + Seasonal + Random
*      	Seasonal changes in the data stay roughly the same over time and don’t fluctuate in relation to the overall data.

* Chart shows a slight decrease in crime rates contributed from seasonality. 



****

# Additional Model
TRYING KMEANS ON TOP 20 percent OF CRIME TYPES 
* This activity is to see if there are any clusters of top 20 percent of crimes that can relate to weeks of the year 

In [ ]:
District = df1['DISTRICT'].value_counts(dropna=np.False_)


In [ ]:
District = pd.DataFrame(District)
District.reset_index(inplace=True)


In [ ]:
District = District.rename(columns={'index':'district'})
District = District.rename(columns={'DISTRICT':'count'})

In [ ]:
Offense_type = df1['OFFENSE_CODE_GROUP'].value_counts(dropna=np.False_)

In [ ]:
Offense_type = pd.DataFrame(Offense_type)
Offense_type.reset_index(inplace=True)


In [ ]:
Offense_type = Offense_type.rename(columns={'index':'Offense'})
Offense_type = Offense_type.rename(columns={'OFFENSE_CODE_GROUP':'count'})

In [ ]:
 Offense_type = Offense_type.rename(columns={'Offense_type':'count'})

### Breakdown of Districts and Offense Types

In [ ]:
# create a groupby DataFrame by DISTRICT
DISTRICT_pivot = df1.groupby(["DISTRICT"]).size()\
.sort_values(ascending = False).to_frame().reset_index().rename(columns = {0: "offenses_per_district"})

DISTRICT_pivot["cumsum_by_district"] = (DISTRICT_pivot["offenses_per_district"]/DISTRICT_pivot["offenses_per_district"].sum()).cumsum()

DISTRICT_pivot["pct_district"] = (1/DISTRICT_pivot.shape[0])
DISTRICT_pivot["pct_district"] = DISTRICT_pivot["pct_district"].cumsum()
DISTRICT_pivot.head()

In [ ]:
# create a groupby DataFrame by OFFENSE TYPE
OFFENSE_pivot = df1.groupby(["OFFENSE_CODE_GROUP"]).size()\
.sort_values(ascending = False).to_frame().reset_index().rename(columns = {0: "offenses_per_group"})

OFFENSE_pivot["cumsum_by_type"] = (OFFENSE_pivot["offenses_per_group"]/OFFENSE_pivot["offenses_per_group"].sum()).cumsum()

OFFENSE_pivot["pct_type"] = (1/OFFENSE_pivot.shape[0])
OFFENSE_pivot["pct_type"] = OFFENSE_pivot["pct_type"].cumsum()
OFFENSE_pivot.head()

In [ ]:
#----------------------
# Plotting part

# Instantiate the figure
fig = plt.figure(figsize = (10, 15))
ax1, ax2, ax3 = fig.subplots(nrows = 3, ncols = 1)

# get the data
# concate a zero before the list, so that all curves start at origin
x1_values = [0] + list(DISTRICT_pivot.index)
y1_values = [0] + list(DISTRICT_pivot["cumsum_by_district"])

x2_values = [0] + list(OFFENSE_pivot["cumsum_by_type"])
y2_values = [0] + list(OFFENSE_pivot["cumsum_by_type"])

x3_values_District = [0] + list(DISTRICT_pivot["pct_district"])
y3_values_District = [0] + list(DISTRICT_pivot["cumsum_by_district"])

x3_values_type = [0] + list(OFFENSE_pivot["pct_type"])
y3_values_type = [0] + list(OFFENSE_pivot["cumsum_by_type"])

# plot the values and set for every subplot a title
ax1.plot(y1_values)
ax1.title.set_text("Percentage of Accumulated Crime counts  by District")

ax2.plot(y2_values, color = "green", alpha = 0.5)
ax2.title.set_text("Percentage of Accumulated Crime counts  by Offense Type")

ax3.plot(x3_values_District, y3_values_District, label = "Percentage of Accumulated Crime counts  by District")
ax3.plot(x3_values_type, y3_values_type, label = "Percentage of Accumulated Crime counts  by Offense Type", 
         color = "green", alpha = 0.5)
ax3.title.set_text("Percentage of Accumulated Crime counts  by District and Offense Type")

ax3.legend()

# create a title for the figure
fig.suptitle('Cumulative distribution of Crime counts by District and Offense Type (absolute and relative)', fontsize = 16);

Crimes per district don't show q significant concentration in one are

Crimes by Type shows a significant concentration in one are - Based on pareto principles a in depth look can be done on 20 percent of the problem

# KMEANS CLUSTERING DATA focusing on 20 percent the problem - Crime Type Group 

In [ ]:
df1.head()

In [ ]:
# create a DataFrame

data_pivot = df1.pivot_table(index='Weekofyear', columns='DISTRICT',
  aggfunc=len, fill_value=0)
data2_pivot = df1.pivot_table(index='Weekofyear', columns='OFFENSE_CODE_GROUP',
  aggfunc=len, fill_value=0)
all_data = pd.merge(data_pivot, data2_pivot, on = ["Weekofyear"] )

In [ ]:
all_data = pd.DataFrame(all_data)

In [ ]:
all_data.head()

In [ ]:
#Flattten multiple index pivots
all_data.columns = all_data.columns.to_flat_index().str.join('_')
all_data.reset_index(inplace=True)

In [ ]:
all_data.columns

In [ ]:
district_clusters = all_data[['YEAR_Motor Vehicle Accident Response','YEAR_Larceny','Weekofyear']]
district_clusters.columns = district_clusters.columns.str.replace('YEAR_Motor Vehicle Accident Response', 'Vehicle_Accident_Response')
district_clusters.columns = district_clusters.columns.str.replace('YEAR_Larceny', 'Larceny')

In [ ]:
district_clusters = district_clusters.drop([52])
district_clusters.tail()

In [ ]:
district_clusters.drop(district_clusters.index[district_clusters['Weekofyear'] == 53], inplace=True)

In [ ]:
fig = px.scatter(x = district_clusters['Vehicle_Accident_Response'], y = district_clusters['Larceny'], title = 'Original Variables' )
fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.update_layout(
    autosize=False,
    width=800,
    height=800)

In [ ]:
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(district_clusters.to_numpy()) 

In [ ]:
scaled_data[0:4]

In [ ]:
fig = px.scatter(x = scaled_data[:,0], y = scaled_data[:,1], title = 'Standarised Variables' )
fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)

In [ ]:
clusters = 3

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=clusters)

In [ ]:
kmeans.fit(scaled_data)

In [ ]:
prediction = kmeans.predict(scaled_data)

In [ ]:
fig = px.scatter(x = scaled_data[:,0], y = scaled_data[:,1], color = prediction, title = 'Scaled Variables' )
fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)

In [ ]:
fig = px.scatter(x =  district_clusters['Vehicle_Accident_Response'], y =  district_clusters['Larceny'], color = prediction, title = 'Original Variables' )
fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)

In [ ]:
fig2 = px.scatter(pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_)), x=0, y=1, title='Cluster Centers')
fig2.update_layout(
    autosize=False,
    width=800,
    height=800,)

## Interpret the result


***

In [ ]:
kmeans.cluster_centers_

In [ ]:
inversed = scaler.inverse_transform(kmeans.cluster_centers_)
inversed

In [ ]:
fig = px.scatter(x = scaled_data[:,0], y = scaled_data[:,1], color = prediction )
fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig2 = px.scatter(pd.DataFrame(kmeans.cluster_centers_), x=0, y=1)
fig2.update_traces(marker=dict(size=20, color='pink', line=dict(width=2, color='white')), selector=dict(mode='markers'), marker_symbol = 'x')

fig.add_trace(fig2.data[0])
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig

In [ ]:
fig = px.scatter(x =  district_clusters['Vehicle_Accident_Response'], y =  district_clusters['Larceny'], color = prediction )
fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig2 = px.scatter(pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_)), x=0, y=1)
fig2.update_traces(marker=dict(size=20, color='pink', line=dict(width=2, color='white')), selector=dict(mode='markers'), marker_symbol = 'x')

fig.add_trace(fig2.data[0])
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig

## Apply to new data

In [ ]:
kmeans.predict([[1,2,3]])

In [ ]:
x1 = 700
y1 = 700
z1 = 700
kmeans.predict(scaler.transform([[x1,y1,z1]])) 

In [ ]:
district_clusters.head()

In [ ]:
fig = px.scatter(x =  district_clusters['Vehicle_Accident_Response'], y =  district_clusters['Larceny'], color = prediction )
fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig2 = px.scatter(pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_)), x=0, y=1)
fig2.update_traces(marker=dict(size=20, color='pink', line=dict(width=2, color='white')), selector=dict(mode='markers'), marker_symbol = 'x')
fig.add_shape(type="circle", xref="x", yref="y", x0=x1+0.1, y0=y1+0.15, x1=x1, y1=y1, line_color="green",fillcolor = 'green')


fig.add_trace(fig2.data[0])
fig.update_layout(template = 'plotly_white')
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig

## Evaluation 

In [ ]:
fig = px.scatter(x =  district_clusters['Vehicle_Accident_Response'], y =  district_clusters['Larceny'], color = prediction, symbol = district_clusters['Weekofyear'] )
fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig2 = px.scatter(pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_)), x=0, y=1)
fig2.update_traces(marker=dict(size=20, color='pink', line=dict(width=2, color='white')), selector=dict(mode='markers'), marker_symbol = 'x')
fig.add_shape(type="circle", xref="x", yref="y", x0=x1+0.1, y0=y1+0.15, x1=x1, y1=y1, line_color="green",fillcolor = 'green')
fig.update_layout(legend_orientation="h")


fig.add_trace(fig2.data[0])
fig.update_layout(template = 'plotly_white')
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig

# CONCLUSION
* CLUSTERS in Yellow are TOP Spikes for both Larceny and Vehicle Assistance Response happening on the following weeks : 25 to 35.

****